In [1]:
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp7hv9zn7b".


In [5]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>

#define TAMANHO 3

__global__ void multiplicaMatriz(int* matrizA, int* matrizB, int* matrizC, int tamanho) {
    int i = blockIdx.y * blockDim.y + threadIdx.y;
    int j = blockIdx.x * blockDim.x + threadIdx.x;
    int soma = 0;

    if (i < tamanho && j < tamanho) {
        for (int tam = 0; tam < tamanho; tam++) {
            soma = soma + matrizA[i * tamanho + tam] * matrizB[tam * tamanho + j];
        }
        matrizC[i * tamanho + j] = soma;
    }
}

void criaMatriz(int* matriz, int tamanho) {
    for (int i = 0; i < tamanho * tamanho; i++) {
        matriz[i] = rand() % 10;
    }
}

void imprimeMatriz(int* matriz, int tamanho) {
    for (int i = 0; i < tamanho; i++) {
        for (int j = 0; j < tamanho; j++) {
            printf("%d ", matriz[i * tamanho + j]);
        }
        printf("\n");
    }
}

int main() {
    srand(time(NULL));
    int size = TAMANHO * TAMANHO * sizeof(int);

    int* h_A = (int*)malloc(size);
    int* h_B = (int*)malloc(size);
    int* h_C = (int*)malloc(size);

    criaMatriz(h_A, TAMANHO);
    criaMatriz(h_B, TAMANHO);

    printf("Matriz A aleatória:\n");
    imprimeMatriz(h_A, TAMANHO);

    printf("\nMatriz B aleatória:\n");
    imprimeMatriz(h_B, TAMANHO);

    int* d_A;
    int* d_B;
    int* d_C;
    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C, size);

    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(TAMANHO, TAMANHO);
    dim3 blocksPerGrid(1, 1);

    multiplicaMatriz<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, TAMANHO);
    cudaDeviceSynchronize();

    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    printf("\nMatriz C - Resultado:\n");
    imprimeMatriz(h_C, TAMANHO);

    // Liberando memória
    free(h_A);
    free(h_B);
    free(h_C);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}


Matriz A aleatória:
6 6 0 
6 2 4 
1 4 7 

Matriz B aleatória:
1 0 7 
5 4 5 
0 9 6 

Matriz C - Resultado:
36 24 72 
16 44 76 
21 79 69 

